# Halite IV - Swarm of Bots (Silver Submission)

Developed towards Halite IV Competition in Kaggle.

Development Started : 2020-07-19 14:52:19 IST

Competition End date: 2020-09-22 11:59 UST

Author : [ARUN P. R.](mailto:arunprathap@gmail.com)

I am sharing my experience and code submitted to Halite IV Competition sponsored by Two Sigma hosted in Kaggle platform. This is the first Featured competition in Kaggle in which I am participating and I am very glad to reach the silver line. There were many ups and downs in the journey, but the end is sweet as always.

### Snapshots

[One of my favourite episodes where I beat opponents with skill level above me.](https://www.kaggle.com/c/halite/leaderboard?dialog=episodes-episode-3485447)

[Another one where I beat higher level submissions of opponents in Top 15](https://www.kaggle.com/c/halite/leaderboard?dialog=episodes-episode-3476886)

## Introduction

### How I came to Kaggle?

I have programming experience in C++, Java, etc through my B.Tech / Masters of Computer Applications courses and as part of my job at VSSC/ISRO. 

On a quest to acquire new skills, I started doing online courses in Machine Learning and Python during the CoViD '19 Lockdown period. And I earned quite a number of [IBM Digital Credentials](https://www.youracclaim.com/users/arun-p-r/badges) and specializations/certificates. 

I was eager to get some hands-on experience and my colleague Kishan Kumar Mishra suggested Kaggle to me. It was an excellent suggestion as with each day since I joined Kaggle, I am learning new things that I couldn't find in any course. The exposure to ideas is mindblowing and I am overwhelmed with new techniques and models. 

Currently I am working on implementing a Transformer based model for Stanford OpenVaccine Challenge in Kaggle and it is driving me nuts by each passing day ... (Since I prefer build from scratch method, not because there are not enough resources available, but that way I will learn more -- Though I copy baselines in competitions I am not serious about like I did for Cornell Birdcall Competition that ended last week).

### How I met Halite?

Titanic was my starting point as it is the place where most Kagglers start their journey. And these diamonds used to pop up in my homepage as simulation competition Halite (I didn't realise they were spaceships at first) and I ignored them for a month thinking the competition is all about Reinforcement Learning. 

And I was thinking like Simulation Competitions can wait till I complete some course in Reinforcement Learning. Then one fine morning, I decided to explore all currently active competitions in Kaggle and to my surprise found that Halite is very much similar to the maze solving robots that we used to develop while in college. And then I started mining Halite....

I decided Halite is my Python hands-on and being my first competition, I decided to go ahead with rule based programming and my target was a code that will be as good as the code I write in C++/Java. And definitely I am not satisfied in this aspect as my code still has many bugs/leakages/inefficient loops which prompts me to update my Python skills and I have shortlisted a few next level courses for perusal.

### Progression in Halite

I started with the Starter code provided for the competition and with each improvement(?) that I did, my score came down by around 50 per submission consistently till my score reached around 300. Then I started getting the hang of things and score started improving by 100 per day till 800. Then it took me a week to cross 900, 2 days to cross 1000, 2 weeks to cross 1100 and another week to cross 1200. 1230 (for 1 week approx with rank 35) was my best score in Halite IV. 

I did a total of about 190 submissions with 131 major versions and others being minor versions for changes. At the end of submission deadline, my best score was around 1190 (ranked 42) and I had multiple copies of my 3 best submission (around 3 each of them) fighting among each other for becoming my best score in the final week. 

With two days to go, my best submission is at rank 40 with the three best submissions scoring 1208, 1196 and 1189 and with current ratings my final rank will be in 38-43 range.

### Strategy : Tends to None

Be optimistic when you are winning and be pessimistic when you are losing. Do the best that you can. Get as much halite as you can, spawn as many ships as you can if you are winning, make ships lightweight in the blitz and hope for the best. 

This particular "strategy of having no strategy" evolved from my Evolutionary game theory based Bourgeoisie strategy (major versions 100-131) submissions scoring well below my old submission 47. So I went back to my submission 47 with two weeks left in the competition and started applying bux fixes and improvements. Finally with 21 minor versions in submission 47, my best bots were 47.0 (1196 bot), 47.12 (1208 bot) and 47.20 (1189 bot). In the last two days, I submitted multiple copies of these three bots which was a good move from the results as I can see.

## Submission File

submission.py is the output file that is to contain the definition of the agent that will be deployed by the halite engine hosted in kaggle. The code is kept as it is in the final submission and no clean up is done and commented portions/bugs are retained as it is.

The starter code used is from my own [baseline](https://www.kaggle.com/arunprathap/getting-started-with-halite-bot-in-a-class-boile). I wanted the code to be less than 1k lines, but final SLOC is ~1.5k.

### Import Environment for submission file and setup context

In [ ]:
%%writefile submission.py

# Imports helper functions
from kaggle_environments.envs.halite.helpers import *
from random import choice, randint
import numpy as np
import pandas as pd
import scipy.ndimage
import scipy.optimize

# Directions a ship can move
directions = [ShipAction.NORTH, ShipAction.EAST, ShipAction.SOUTH, ShipAction.WEST]
init_Controller = False
controller = None

## Define the Controller Class

### Start by initializing class variables and constants

In [ ]:
%%writefile -a submission.py

class Controller():
        
    def __init__(self, obs, config):
        
        #Initialise Instance Variables
        self.size = config.size
        self.board = Board(obs, config)  
        self.me = self.board.current_player
        self.episode_steps = self.board.configuration.episode_steps
        self.convert_cost = self.board.configuration.convert_cost
        self.spawn_cost = self.board.configuration.spawn_cost
        self.HALITE_THRESHOLD = 50
        self.HALITE_THRESHOLD_MIN = 50
        self.REUSE_TARGET_LIMIT = 1
        self.MIN_SHIPS = 5
        #self.RECURSION_LIMIT = 2
        #self.STATIONARY_SHIPS_LIMIT = 2
        
        #Lookup Table
        self.reference_distance = {}
        self.create_distance_lookup()        

        #self.generate_farming_map()

### Define Next Actions for the controller

In [ ]:
%%writefile -a submission.py

    def next_actions(self, obs, config):
    
        #Initialise Context Variables
        self.board = Board(obs, config)
        self.me = self.board.current_player
        self.ships = self.me.ships
        self.shipyards = self.me.shipyards
        self.ship_ids = self.me.ship_ids
        self.shipyard_ids = self.me.shipyard_ids

        self.enemy_ships = []
        self.enemy_shipyards = []
        
        self.max_enemy_ships = 0
        self.max_enemy_halite = 0

        #Initialise Loop Variables
        self.remaining_ship_count = len(self.ships)
        self.remaining_shipyard_count = len(self.shipyards)
        self.remaining_halite = self.me.halite
        self.halite_in_ships = 0
        self.overwhelming_ship_spawn = False
        #self.too_many_ships_onboard = False
        #if self.remaining_ship_count <= len(self.board.ships) // 5 :
        #    self.too_many_ships_onboard = True
        
        for ship in self.ships:
            self.halite_in_ships += ship.halite
        
        zero_ships = 0
        
        for player in self.board.opponents:
            enemy_halite = player.halite
            self.enemy_ships.extend(player.ships)
            self.enemy_shipyards.extend(player.shipyards)
            for ship in player.ships:
                enemy_halite += ship.halite
                if ship.halite <= 0.0 :
                    zero_ships += 1
            if enemy_halite >= (self.remaining_halite + self.halite_in_ships) :
                self.max_enemy_ships = max(self.max_enemy_ships, len(player.ships))
            self.max_enemy_halite = max(self.max_enemy_halite, enemy_halite)
            #if len(player.ships) >= len(self.board.ships) // 2 :
            #    self.too_many_ships_onboard = True
            #if len(player.ships) >= 2 * self.remaining_ship_count : 
            #    self.too_many_ships_onboard = True  
            #if len(player.ships) >= 1.5 * self.remaining_ship_count : 
            #    self.too_many_ships_onboard = True                
      
        if zero_ships >= len(self.board.ships) // 2 :
            self.overwhelming_ship_spawn = True
        if zero_ships >= 2 * self.remaining_ship_count : 
            self.overwhelming_ship_spawn = True   
        #if zero_ships >= self.remaining_ship_count : 
        #    self.too_many_ships_onboard = True             
        #self.recursion_level = 0
        #self.stationary_ships = 0           
        self.assigned_mining_targets = {}        
    
        self.update_swarm_info()
        
        self.update_halite_map()
        self.minimum_ships = 5
        if self.board.step <= self.episode_steps // 4:
            self.minimum_ships = 13
        elif self.board.step <= self.episode_steps * 5 // 8 and self.board.step > self.episode_steps // 4:
            self.minimum_ships = 9
        elif self.board.step <= self.episode_steps * 3 // 4 and self.board.step > self.episode_steps * 5 // 8:
            self.minimum_ships = 9
        elif self.board.step <= self.episode_steps * 7 // 8 and self.board.step > self.episode_steps * 3 // 4:
            self.minimum_ships = 9
        
        self.MIN_SHIPS = self.minimum_ships
        #self.MIN_SHIPS = max(self.minimum_ships, self.max_enemy_ships + 1)#, self.remaining_ship_count)
        #self.MIN_SHIPS = max(9, self.remaining_ship_count)
        self.ship_next_actions()
        self.shipyard_next_actions()
        #print(self.board.step, self.me.id, self.me.next_actions)
        return self.me.next_actions

### Ship Next Actions

In [ ]:
%%writefile -a submission.py

    def ship_next_actions(self):

        # no ships left return
        if self.remaining_ship_count == 0:
            return

        # try to find allowed actions for each ship
        # self ships are not taken in to consideration
        # must be considered in iteration
        self.optimize_ship_action_matrix()

        if self.start_end_move:
            self.ship_action_end_move()  
            
        # allow only single convert in steps other than towards game end
        # to have shipyards atleast 10 steps or so apart to have good board coverage
        self.allow_convert = True
        
        #else:
        #    self.convert_candidates = sorted(self.convert_candidates, key=self.convert_candidates.get, reverse = True)            
        #    if self.allow_convert :                
        #        for ship in self.convert_candidates:
                    #if ship.halite >= 2 * self.convert_cost:
        #            if self.ship_action_convert(ship):
        #                self.allow_convert = False
        #                break                     
        
        # if all ships were converted no need to proceed further in this function
        #if self.remaining_ship_count == 0:
        #    return
        
        ship_list = self.find_best_ship_to_convert()
        # 100 to 150 to see if my ships can offload halite to shipyard
        # i have a shipyard but i am not allowed to go near it
        # try one ship at a time
        if self.allow_convert and self.board.step >= 150 and\
           self.halite_in_ships >= self.remaining_ship_count * self.convert_cost:
            # i am not being allowed to offload to shipyard
            # sacrifice my ship and enemy ship for one additional ship
            # ideally one with max halite            
            for ship in ship_list:
                if self.is_not_none(ship) and ship.halite >= 2 * self.convert_cost:
                    if self.ship_action_convert(ship):
                        self.allow_convert = False
                        break
                        
        # no shipyards left and i may have multiple ships to choose from               
        if self.allow_convert and self.remaining_shipyard_count == 0 and \
           (self.should_i_spawn() or self.board.step >= (self.episode_steps - self.size - 2)  or\
            self.halite_in_ships >= max(20 * self.HALITE_THRESHOLD, self.remaining_halite//10)):
            for ship in ship_list:
                if self.is_not_none(ship):
                    if self.ship_action_convert(ship):
                        self.allow_convert = False
                        break
                        
        # end of game is near
        if self.board.step == (self.episode_steps - 2) or self.convert_now():
            for ship in self.ships:
                if ship.halite > self.convert_cost:
                    # check here for conversion status so that if any adjacent ship wants to drop halite they can do so
                    if self.ship_action_convert(ship):
                        continue
        
        

### Shipyard Next Actions

In [ ]:
%%writefile -a submission.py

    def shipyard_next_actions(self):
            
        # do not spawn new ships towards competition end
        if (self.convert_now() or self.board.step > self.episode_steps - self.size):
            return
            
        for shipyard in self.shipyards:
            
            # there is a ship in shipyard and it is staying
            if self.is_not_none(shipyard.cell.ship) and self.is_none(shipyard.cell.ship.next_action):
                continue
            
            #protect shipyard by spawning ship and sacrificing if required
            if (self.is_none(shipyard.cell.ship) and \
               len(self.get_enemy_ships(self.get_adjacent_ships(shipyard.cell))) != 0):
                #if self.remaining_halite >= self.spawn_cost * (self.remaining_ship_count + self.remaining_shipyard_count) and self.remaining_halite >= halite_in_ships:
                if self.shipyard_action_spawn(shipyard):
                    continue
                    
            # if concentration of ships around this shipyard is high go to other shipyards for spawning
            if self.remaining_shipyard_count > 1:
                count = 0
                limit = self.remaining_ship_count // self.remaining_shipyard_count + 1
                for my_ship in self.ships:
                    if self.get_nearest_entity(my_ship.position, self.shipyard_ids) == shipyard:
                        count += 1
                    if count > limit:
                        break
                if count > limit:
                    continue            
            
            #if self.board.step < 11:
            if self.should_i_spawn():
                if self.shipyard_action_spawn(shipyard):
                    continue      

### Shipyard and Ship Actions

In [ ]:
%%writefile -a submission.py

    def shipyard_action_spawn(self, shipyard):
        
        protect_spawn = True
        adjacent_ships = self.get_adjacent_ships(shipyard.cell)
        
        for my_ship in self.get_my_ships(adjacent_ships):
            if my_ship.next_action == self.get_dir_to(my_ship.position, shipyard.position):
                protect_spawn = False
                break

        for enemy_ship in self.get_enemy_ships(adjacent_ships):
            if enemy_ship.halite <= 0.0:
                protect_spawn = False
                break
                
        if protect_spawn :        
            if self.remaining_halite >= self.spawn_cost:
                shipyard.next_action = ShipyardAction.SPAWN
                self.remaining_ship_count += 1
                self.remaining_halite -= self.spawn_cost    
                return True  
            
        return False 

In [ ]:
%%writefile -a submission.py

    def ship_action_convert(self, ship):
        # I should not convert a ship if I am in a shipyard
        if self.is_not_none(ship.cell.shipyard):
            return False
        if (self.remaining_halite + ship.halite) >= self.convert_cost:
            ship.next_action = ShipAction.CONVERT
            self.remaining_ship_count -= 1
            self.remaining_shipyard_count += 1
            self.remaining_halite += ship.halite
            self.remaining_halite -= self.convert_cost    
            return True
        return False

In [ ]:
%%writefile -a submission.py

    def find_best_ship_to_convert(self):

        first_choice = {}
        second_choice = {}
        ship_list = []
        
        if len(self.convert_candidates) != 0:
            self.convert_candidates = sorted(self.convert_candidates, key = self.convert_candidates.get,\
                                             reverse = True)
            ship_list.extend(self.convert_candidates)
        
        for ship in self.ships:
            if ship not in ship_list:
                if not self.ship_actions_bitfield[ship]['pirate']:
                    first_choice[ship] = ship.halite
                else:
                    second_choice[ship] = ship.halite
                                
        if len(first_choice) != 0:
            first_choice = sorted(first_choice, key = first_choice.get, reverse = True)
            ship_list.extend(first_choice)
        
        if len(second_choice) != 0:
            second_choice = sorted(second_choice, key = second_choice.get, reverse = True)
            ship_list.extend(second_choice)
        
        return ship_list

## Decisions that will decide whether you win/lose

In [ ]:
%%writefile -a submission.py
    
    def should_i_spawn(self):
        
        if (self.convert_now() or self.board.step > self.episode_steps - 2 * self.size):
                # do not spawn new ships towards competition end
                return False
        
        #if self.reached_base:
        if self.board.step > 11:
            if self.remaining_ship_count > self.MIN_SHIPS:
                if self.remaining_ship_count > len(self.board.ships) // 2 + 2 and self.remaining_halite <= self.max_enemy_halite:
                    return False
                if self.remaining_halite < 2 * self.convert_cost:
                    return False
                if self.remaining_ship_count < len(self.board.ships) // 5 and self.board.step <= self.episode_steps*3//4:
                    return True    
                if self.board.step <= self.episode_steps//4 and ((self.remaining_halite + self.halite_in_ships) <= (self.max_enemy_halite + self.convert_cost)):# or\
               #self.remaining_ship_count > (len(self.board.ships) // 2) or\
               #self.remaining_ship_count > self.max_enemy_ships):
                    return False
                elif self.board.step <= self.episode_steps//2 and ((self.remaining_halite + self.halite_in_ships) <= (self.max_enemy_halite + 1.5 * self.convert_cost)):# or\
                    return False
                elif self.board.step <= self.episode_steps*3//4 and ((self.remaining_halite + self.halite_in_ships) <= (self.max_enemy_halite + 2 * self.convert_cost)):# or\
                    return False
                elif self.board.step <= self.episode_steps*7//8 and ((self.remaining_halite + self.halite_in_ships) <= (self.max_enemy_halite + 3 * self.convert_cost)):# or\
                    return False
                elif self.board.step > self.episode_steps*7//8 and (self.remaining_halite <= (self.max_enemy_halite + self.convert_cost)):# or\
                    return False

            #elif self.board.step >= 200 and self.board.step <= 350 and self.remaining_ship_count > max(3 * (self.MIN_SHIPS - 1), len(self.shipyards) * self.MIN_SHIPS):
            #    return False        
            #if self.remaining_ship_count <= 3 * (self.MIN_SHIPS * len(self.shipyards) - 1) or\
             #  (self.remaining_ship_count <= min(4 * self.MIN_SHIPS * len(self.shipyards), self.max_enemy_ships) and\
              #  self.HALITE_THRESHOLD >= 2 * self.HALITE_THRESHOLD_MIN) :
               # self.remaining_ship_count <= self.max_enemy_ships and\
               # self.HALITE_THRESHOLD >= self.HALITE_THRESHOLD_MIN) or\
               #(self.remaining_ship_count <= 4 * self.MIN_SHIPS and\
               # self.remaining_ship_count <= self.max_enemy_ships and\
               # self.HALITE_THRESHOLD >= 2 * self.HALITE_THRESHOLD_MIN) or\
               #(self.remaining_ship_count <= 5 * self.MIN_SHIPS * len(self.shipyards) - 1 and\
               # self.HALITE_THRESHOLD >=  3 * self.HALITE_THRESHOLD_MIN):
                #return True
        #else:
        #    return False
        return True

In [ ]:
%%writefile -a submission.py
    
    def should_i_move_towards_shipyard(self, ship):
        #if self.should_i_spawn and self.remaining_halite <= self.convert_cost:
         #   if ship.halite >= 2 * self.convert_cost:
          #      return True 
        #else:
        if self.remaining_shipyard_count == 0:
            return False        
        level = 0
        if (self.HALITE_THRESHOLD <=  self.HALITE_THRESHOLD_MIN):
            level = 1
        elif (self.HALITE_THRESHOLD > self.HALITE_THRESHOLD_MIN and\
            self.HALITE_THRESHOLD <= 2 * self.HALITE_THRESHOLD_MIN):
            level = 2
        elif (self.HALITE_THRESHOLD > 2 * self.HALITE_THRESHOLD_MIN and\
            self.HALITE_THRESHOLD <= 3 * self.HALITE_THRESHOLD_MIN):
            level = 3
        elif (self.HALITE_THRESHOLD > 3 * self.HALITE_THRESHOLD_MIN and\
            self.HALITE_THRESHOLD <= 4 * self.HALITE_THRESHOLD_MIN):
            level = 4
        elif (self.HALITE_THRESHOLD > 4 * self.HALITE_THRESHOLD_MIN and\
            self.HALITE_THRESHOLD <= 5 * self.HALITE_THRESHOLD_MIN):
            level = 5
        elif (self.HALITE_THRESHOLD > 5 * self.HALITE_THRESHOLD_MIN and\
            self.HALITE_THRESHOLD <= 6 * self.HALITE_THRESHOLD_MIN):
            level = 6
        elif  (self.HALITE_THRESHOLD > 6 * self.HALITE_THRESHOLD_MIN):    
            level = 7

        if self.overwhelming_ship_spawn:
            level = -1
        elif self.episode_steps - self.board.step <= self.size + 2:
            level = 0
        #elif self.too_many_ships_onboard:
        #    level = 0
        elif self.remaining_halite < 2 * self.convert_cost and\
             ((self.board.step >= self.episode_steps // 5 and ship.halite >= 2 * self.convert_cost // self.remaining_ship_count) or\
              (self.board.step < self.episode_steps // 5 and ship.halite >= 1.5 * self.convert_cost)):
            return True
        elif self.board.step < self.episode_steps // 5:
            level = 7
        elif len(self.board.ships) < 4 * self.MIN_SHIPS:
        #    factor = (400 - self.board.step) // self.size
            level = 6
        #elif self.should_i_spawn():
        #    level -= 1
    #ship.halite >= min(self.HALITE_THRESHOLD, self.HALITE_THRESHOLD_MIN) and level <= 0
        if (ship.halite > 0.0 and level < 0) or\
           (ship.halite >= min(self.HALITE_THRESHOLD, self.HALITE_THRESHOLD_MIN) and level == 0) or\
           (ship.halite >= 1 * self.HALITE_THRESHOLD_MIN and level == 1) or\
           (ship.halite >= 2 * self.HALITE_THRESHOLD_MIN and level == 2) or\
           (ship.halite >= 4 * self.HALITE_THRESHOLD_MIN and level == 3) or\
           (ship.halite >= 8 * self.HALITE_THRESHOLD_MIN and level == 4) or\
           (ship.halite >= 16 * self.HALITE_THRESHOLD_MIN and level == 5) or\
           (ship.halite >= 32 * self.HALITE_THRESHOLD_MIN and level == 6) or\
           (ship.halite >= 64 * self.convert_cost and level >= 7):
            return True
        return False

In [ ]:
%%writefile -a submission.py
    
    def am_i_powerful(self, ship, list_of_ships, consider_neighbours = True, skip_conflict = False):
        i_am_powerful = True
        strongest_ship = ship
        conflict = False
        for enemy_ship in self.get_enemy_ships(list_of_ships):
            if enemy_ship.halite < ship.halite:
                strongest_ship = enemy_ship
                i_am_powerful = False
                break
            elif enemy_ship.halite == ship.halite:
                i_am_powerful = False
                conflict = True
        if consider_neighbours:
            if not i_am_powerful and len(self.get_my_ships(list_of_ships)) != 0:
                # if i have an adjancent self ship which is the strongest of the lot,
                # i am still powerful
                # initialise with weaker ship ie me
                for other_ship in list_of_ships:
                    if other_ship.halite < strongest_ship.halite:
                        strongest_ship = other_ship
                        conflict = False
                    elif other_ship.halite == strongest_ship.halite:
                        if other_ship.player != strongest_ship.player:
                            conflict = True
                        if other_ship.player == ship.player:
                            strongest_ship = other_ship
        if strongest_ship.player == ship.player and (not conflict or skip_conflict):
            i_am_powerful = True
        return i_am_powerful

In [ ]:
%%writefile -a submission.py
    
    def optimize_ship_action_matrix(self):
        
        if self.remaining_ship_count == 0:
            return
        
        self.start_end_move = False
        
        self.position_entries = []
        self.ship_actions_bitfield = {}
        self.convert_candidates = {}
        should_i_spawn = self.should_i_spawn()
        
        for ship in self.ships: 
            
            self.ship_actions_bitfield[ship] = {}
            self.ship_actions_bitfield[ship]['actions'] = {}
            self.ship_actions_bitfield[ship]['actions'][ship.position] = {}
            self.ship_actions_bitfield[ship]['actions'][ship.position]['action'] = None            
            self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] = 0
            self.ship_actions_bitfield[ship]['pirate'] = False
            self.ship_actions_bitfield[ship]['convert_candidate'] = True
            self.ship_actions_bitfield[ship]['assigned'] = True
            
            should_i_move_towards_shipyard = self.should_i_move_towards_shipyard(ship)

            nearest_shipyard, distance_nearest_shipyard, direction1_nearest_shipyard, \
            direction2_nearest_shipyard = self.get_nearest_shipyard_details(ship, self.shipyard_ids)
            
            self.ship_actions_bitfield[ship]['nearest_shipyard'] = {}
            self.ship_actions_bitfield[ship]['nearest_shipyard']['shipyard'] = nearest_shipyard
            self.ship_actions_bitfield[ship]['nearest_shipyard']['distance'] = distance_nearest_shipyard
            self.ship_actions_bitfield[ship]['nearest_shipyard']['direction1'] = direction1_nearest_shipyard
            self.ship_actions_bitfield[ship]['nearest_shipyard']['direction2'] = direction2_nearest_shipyard
            
            swarm_shipyard, ship_index, am_i_guard = self.get_swarm_shipyard_details(ship, nearest_shipyard)

            if self.am_i_powerful(ship, self.get_adjacent_ships(ship.cell)):
                self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] = 500
            else:
                self.ship_actions_bitfield[ship]['convert_candidate'] = False

            for other_ship in self.get_enemy_ships(self.get_adjacent_ships(ship.cell)):
                if other_ship.halite < ship.halite:
                    self.ship_actions_bitfield[ship]['pirate'] = True
                    break

            box_in_possibility_count = 0        
            
            for direction in directions:
                
                dict_direction = {}
                dict_direction['action'] = direction
                dict_direction['cost'] = 0
                
                adjacent_ship_in_direction =  self.get_adjacent_ship_in_direction(ship.cell, direction)
                nearby_ships_in_direction = self.get_nearby_ships_in_direction(ship.cell, direction)
                approaching_ships_in_direction = self.get_approaching_ships_in_direction(ship.cell, direction)
                adjacent_shipyard_in_direction = self.get_adjacent_shipyard_in_direction(ship.cell, direction)
                halite_adjacent_cell_in_direction = self.get_adjacent_halite_in_direction(ship.cell, direction)

                adjacent_nearby_ships = list(nearby_ships_in_direction)
                if self.is_not_none(adjacent_ship_in_direction):
                    adjacent_nearby_ships.append(adjacent_ship_in_direction)
                    
                adjacent_nearby_approaching_ships = self.union_of_lists(adjacent_nearby_ships, \
                                                                        self.get_enemy_ships(approaching_ships_in_direction))
                
                i_am_powerful = self.am_i_powerful(ship, adjacent_nearby_approaching_ships)
                if i_am_powerful:
                    dict_direction['cost'] = 500
                else:
                    self.ship_actions_bitfield[ship]['convert_candidate'] = False
                    i_am_powerful = self.am_i_powerful(ship, adjacent_nearby_ships)
                    if i_am_powerful:
                        dict_direction['cost'] = 500                
                for other_ship in self.get_enemy_ships(approaching_ships_in_direction):
                    if other_ship.halite < ship.halite:
                        dict_direction['cost'] -= 50
                        
                if dict_direction['cost'] < 400:
                    box_in_possibility_count += 1
                    
                if self.am_i_powerful(ship, nearby_ships_in_direction, False):
                    # weak attack on enemy ships
                    if self.is_not_none(adjacent_ship_in_direction) and\
                       self.is_enemy_ship(adjacent_ship_in_direction):
                        if adjacent_ship_in_direction.halite > ship.halite:
                            dict_direction['cost'] += adjacent_ship_in_direction.halite // 50
                            if self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] >= 500:
                                self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += adjacent_ship_in_direction.halite // 50
                        #elif adjacent_ship_in_direction.halite == ship.halite and\
                        #     len(adjacent_ship_in_direction.player.ships) >= self.remaining_ship_count:        
                        #    if self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] >= 500:
                        #        self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += adjacent_ship_in_direction.halite // 50                            
                    # 1/5 since it is 1 out of 5 options, 1/2 because it can be a nearby ship
                    # 1/5 since boxin is 1 out of 5 options of my other ship
                  #  self.stationary_ships += 1
                    
                # may add up to 500/4/2 = 62.5 in this step
                if i_am_powerful and not self.ship_actions_bitfield[ship]['pirate'] and\
                   not am_i_guard and\
                   halite_adjacent_cell_in_direction >= self.HALITE_THRESHOLD // 2:
                    #if not should_i_spawn:
                    #assign = False
                    # maintain quadrants
                    #if halite_adjacent_cell_in_direction >= min(2 * self.HALITE_THRESHOLD_MIN, self.HALITE_THRESHOLD):
                    #    assign = True
                    #elif self.is_not_none(ship_index) and self.is_not_none(swarm_shipyard):

                    #    direction1_check = self.get_dir_to(swarm_shipyard.position,\
                    #                                           (ship.position + direction.to_point())%self.size)
                    #    direction2_check = self.get_dir_to_alt(swarm_shipyard.position,\
                    #                                           (ship.position + direction.to_point())%self.size)

                    #    if (ship_index % 2 == 0):
                    #        if directions[ship_index % len(directions)] == direction1_check and\
                    #               (directions[(ship_index +1) % len(directions)] == direction2_check or\
                    #                directions[ship_index % len(directions)] == direction2_check):
                    #            assign = True
                    #    else:
                    #        if directions[ship_index % len(directions)] == direction2_check and\
                    #               (directions[(ship_index +1) % len(directions)] == direction1_check or\
                    #                directions[ship_index % len(directions)] == direction1_check):
                    #            assign = True
                    #else:
                    #    assign = True 
                    #if assign:
                    dict_direction['cost'] += (halite_adjacent_cell_in_direction // 4 // 2)
                        # //4 since I can mine only 25% in a turn
                        # //2 since I can mine in next step only possibly   

                elif not i_am_powerful :
                    if halite_adjacent_cell_in_direction <= 0.0 and not am_i_guard:
                        dict_direction['cost'] += 10
                    else:
                        dict_direction['cost'] -= halite_adjacent_cell_in_direction

                #if ship.halite <= 0.0 and not self.ship_actions_bitfield[ship]['pirate']:
                #    if self.stationary_ships <= self.STATIONARY_SHIPS_LIMIT and ship.position in self.farming_targets:
                #        self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 100
                #        self.stationary_ships += 1
                        
                if self.is_not_none(adjacent_ship_in_direction) and not am_i_guard:
                    if self.is_enemy_ship(adjacent_ship_in_direction):   
                        # in a shipyard or not
                        if (len(self.get_my_shipyards(self.get_nearby_shipyards(adjacent_ship_in_direction.cell))) != 0 or\
                            len(self.get_my_shipyards(self.get_adjacent_shipyards(adjacent_ship_in_direction.cell))) != 0):
                            if self.am_i_powerful(ship, nearby_ships_in_direction, False) and\
                               adjacent_ship_in_direction.halite <= ship.halite :
                                dict_direction['cost'] += 500

                #don't waste my ship for collision with enemy shipyard
                #can be modified for an offensive version of bot
                if self.is_not_none(adjacent_shipyard_in_direction):
                    if self.is_enemy_shipyard(adjacent_shipyard_in_direction):                        
                        if (len(self.get_my_shipyards(self.get_nearby_shipyards(adjacent_shipyard_in_direction.cell))) != 0 or\
                            len(self.get_my_shipyards(self.get_adjacent_shipyards(adjacent_shipyard_in_direction.cell))) != 0):
                            if (self.is_none(adjacent_shipyard_in_direction.cell.ship) and\
                                (adjacent_shipyard_in_direction.player.halite < self.convert_cost or\
                                self.halite_in_ships >= max(20 * self.HALITE_THRESHOLD, self.remaining_halite//10)) and\
                                self.am_i_powerful(ship, nearby_ships_in_direction, False)):
                                dict_direction['cost'] += 500
                        elif not am_i_guard and ship.halite <= 0.0 and ship.cell.halite <= 0.0 and\
                           len(adjacent_shipyard_in_direction.player.ships) >= self.remaining_ship_count :#check for active player
                            self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 1000
                        #elif ship.halite <= self.HALITE_THRESHOLD and (self.should_i_spawn() or\
                        #     self.remaining_ship_count >= len(self.board.ships) // 2) and\
                        #     len(adjacent_shipyard_in_direction.player.ships) != 0 and\
                        #     adjacent_shipyard_in_direction.player.halite < self.spawn_cost and dict_direction['cost'] > 400:
                        #    dict_direction['cost'] += 500
                        else:
                            dict_direction['cost'] -= (500 + ship.halite)
                        #and not self.ship_actions_bitfield[ship]['pirate']:
                        #    if self.stationary_ships <= self.STATIONARY_SHIPS_LIMIT :
                        #        self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 100
                        #        self.stationary_ships += 1
                    else:
                        # my shipyard
                        #if self.episode_steps  - self.board.step <= self.size and ship.halite > 0.0 and dict_direction['cost'] >= 400:
                        #        dict_direction['cost'] += 500 + ship.halite
                                #if self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] >= 500:
                                #    self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 100 + ship.cell.halite // 4
                        #else:
                        if should_i_spawn and self.remaining_halite >= self.spawn_cost:
                            dict_direction['cost'] -= 500
                            if am_i_guard and self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] >= 400:
                                self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 500
                        else:
                            if dict_direction['cost'] >= 400 :
                                if len(self.get_enemy_ships(self.get_adjacent_ships(adjacent_shipyard_in_direction.cell))) != 0 and\
                                    self.is_none(adjacent_shipyard_in_direction.cell.ship):
                                    dict_direction['cost'] += 500 - ship.halite
                                elif len(self.get_enemy_ships(self.get_nearby_ships(adjacent_shipyard_in_direction.cell))) != 0 :
                                    dict_direction['cost'] += 500 + ship.halite                                       
                                elif len(self.get_enemy_ships(self.get_approaching_ships(adjacent_shipyard_in_direction.cell))) != 0 :
                                    dict_direction['cost'] += 500 + ship.halite
                                elif ship.halite > 0:
                                    dict_direction['cost'] += 500 + ship.halite
                                elif am_i_guard and len(self.get_my_ships(nearby_ships_in_direction)) == 0:
                                     dict_direction['cost'] += 500 + ship.halite
                            elif self.board.step >= self.episode_steps * 7 // 8:
                                # I have to save my shipyard since I dont have enough to make new one
                                if self.is_none(adjacent_shipyard_in_direction.cell.ship) and\
                                   len(self.get_enemy_ships(self.get_adjacent_ships(adjacent_shipyard_in_direction.cell))) != 0 and\
                                   self.am_i_powerful(ship, nearby_ships_in_direction, False, True):
                                    dict_direction['cost'] += (500 - ship.halite)
                            if am_i_guard and\
                               self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] >= 400 and\
                               len(self.get_my_ships(nearby_ships_in_direction)) != 0 and\
                               len(self.get_enemy_ships(self.get_adjacent_ships(adjacent_shipyard_in_direction.cell))) == 0:
                                self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 500
                                
                self.ship_actions_bitfield[ship]['actions'][(ship.position + direction.to_point())%self.size] = dict(dict_direction)
            
            if box_in_possibility_count >= 3:
                self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] -= 100
                
            if self.ship_actions_bitfield[ship]['convert_candidate']:
                # conditions for multiple shipyard
                #if ship.cell.halite < min(2 * self.HALITE_THRESHOLD, self.HALITE_THRESHOLD_MIN) :
                #if self.is_not_none(nearest_shipyard) and self.remaining_ship_count > self.MIN_SHIPS * len(self.shipyards):
                    #if distance_nearest_shipyard >= self.size - 1: #// 2:
                if len(self.get_enemy_shipyards(self.get_adjacent_shipyards(ship.cell))) == 0 and len(self.get_enemy_shipyards(self.get_nearby_shipyards(ship.cell))) == 0:
                    self.convert_candidates[ship] = ship.halite
            
            # adjacent enemy ship -500 already accounted for in case I have any adjacent ship  -->
            # self.ship_actions_bitfield[ship][ship.position]['cost'] = -(500 + ship.halite)
            # add incentive if in shipyard and adjacent or nearby enemy ship
            if self.is_not_none(ship.cell.shipyard):
                # i am guard
                # i have zero halite
                if (should_i_spawn and self.remaining_halite >= self.spawn_cost):
                    self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] -= 500
                    
                    if self.is_not_none(ship_index) and self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[ship_index%len(directions)].to_point())%self.size]['cost'] >= 400:
                        self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[ship_index%len(directions)].to_point())%self.size]['cost'] += 500
                
                else:
                    if self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] >= 400:
                        if len(self.get_enemy_ships(self.get_adjacent_ships(ship.cell))) != 0 :                        
                            self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 500                                
                        elif len(self.get_enemy_ships(self.get_nearby_ships(ship.cell))) != 0 and\
                             len(self.get_my_ships(self.get_adjacent_ships(ship.cell))) == 0:
                            self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 500
                        elif len(self.get_enemy_ships(self.get_approaching_ships(ship.cell))) != 0 and\
                             len(self.get_my_ships(self.get_adjacent_ships(ship.cell))) == 0:
                            self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 500  
                        elif len(self.get_my_ships(self.get_adjacent_ships(ship.cell))) == 0:
                            self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 500
                    elif self.board.step >= self.episode_steps * 7 // 8:
                        if len(self.get_enemy_ships(self.get_adjacent_ships(ship.cell))) != 0 and\
                           self.am_i_powerful(ship, self.get_enemy_ships(self.get_adjacent_ships(ship.cell)), False, True):                        
                            self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 500
                    #elif self.am_i_powerful(ship, self.get_enemy_ships(self.get_adjacent_ships(ship.cell)), False, True):                        
                        # otherwise it will be suicide as I will sacrifice my guard ships one by one
                    #    self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 500
                            
            if self.is_not_none(nearest_shipyard) and self.is_not_none(direction1_nearest_shipyard) and\
               self.is_not_none(direction2_nearest_shipyard) :
                if distance_nearest_shipyard > 1:
                    if distance_nearest_shipyard >= self.episode_steps - self.board.step - 5:            
                        self.start_end_move = True
                        if ship.halite > 0.0:
                            if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] >= 400:
                                self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] += 500 + ship.halite

                            if direction1_nearest_shipyard != direction2_nearest_shipyard: 
                                if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] >= 400:
                                    self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] += 500 + ship.halite
                            elif self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] < 400:
                                self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[(directions.index(direction1_nearest_shipyard) + 1)%len(directions)].to_point())%self.size]['cost'] += 10
                                self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[(directions.index(direction1_nearest_shipyard) + 1)%len(directions)].to_point())%self.size]['cost'] += 10
                        #else:
                        #    self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] -= 500

                        #    if direction1_nearest_shipyard != direction2_nearest_shipyard: 
                        #        self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] -= 500

                        #if self.episode_steps - self.board.step <= self.size + 2 and\
                        #   distance_nearest_shipyard < self.episode_steps - self.board.step - 2 and\
                        #   ship.halite <= 0.0 and\
                        #   ship.cell.halite >= min(self.HALITE_THRESHOLD_MIN, self.HALITE_THRESHOLD):
                        #    if self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] >= 500:
                        #        self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += (ship.cell.halite // 4)
                                # //4 since I can mine only 25% in a turn

                    elif am_i_guard:                    
                        #guard_cost = 0

                        #if len(self.get_enemy_ships(self.get_adjacent_ships(nearest_shipyard.cell))) != 0 or\
                        #   len(self.get_enemy_ships(self.get_nearby_ships(nearest_shipyard.cell))) != 0 or\
                        #   len(self.get_enemy_ships(self.get_approaching_ships(nearest_shipyard.cell))) != 0 :
                        #    guard_cost = 500
                        guard_cost = 500
                        #if not (should_i_spawn and self.remaining_halite >= self.spawn_cost):# and guard_cost == 500:
                        if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] > 0:
                            self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] += (guard_cost + ship.halite) #+ guard_cost

                        if direction1_nearest_shipyard != direction2_nearest_shipyard: 
                            if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] > 0:
                                self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] += (guard_cost + ship.halite)# + guard_cost
                        elif self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] <= 0:
                            self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[(directions.index(direction1_nearest_shipyard) + 1)%len(directions)].to_point())%self.size]['cost'] += 10
                            self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[(directions.index(direction1_nearest_shipyard) - 1)%len(directions)].to_point())%self.size]['cost'] += 10

                        #elif guard_cost == 0:

                        #    if distance_nearest_shipyard > 1:
                         #       if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] >= 400:
                          #          self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] += (500 + ship.halite) #+ guard_cost

                           #     if direction1_nearest_shipyard != direction2_nearest_shipyard: 
                            #        if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] >= 400:
                             #           self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] += (500 + ship.halite)# + guard_cost
                              #  elif self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] < 400:
                               #     self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[(directions.index(direction1_nearest_shipyard) + 1)%len(directions)].to_point())%self.size]['cost'] += 10
                                #    self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[(directions.index(direction1_nearest_shipyard) - 1)%len(directions)].to_point())%self.size]['cost'] += 10

                    elif self.board.step > 11:   

                        if should_i_move_towards_shipyard or self.ship_actions_bitfield[ship]['pirate']:
                            if self.ship_actions_bitfield[ship]['pirate'] or not(self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] >= 500 and ship.cell.halite >= min(2 * self.HALITE_THRESHOLD_MIN, self.HALITE_THRESHOLD)):
                                if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] >= 400:
                                    self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] += 50 + ship.halite // distance_nearest_shipyard

                                if direction1_nearest_shipyard != direction2_nearest_shipyard: 
                                    if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] >= 400:
                                        self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] += 50 + ship.halite // distance_nearest_shipyard
                                elif self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] < 400:
                                    self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[(directions.index(direction1_nearest_shipyard) + 1)%len(directions)].to_point())%self.size]['cost'] += 10
                                    self.ship_actions_bitfield[ship]['actions'][(ship.position + directions[(directions.index(direction1_nearest_shipyard) - 1)%len(directions)].to_point())%self.size]['cost'] += 10

            if not am_i_guard:
                if self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] >= 400 and\
                   ship.cell.halite >= self.HALITE_THRESHOLD//2:
                    self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += ship.cell.halite//4
                    # //4 since I can mine only 25% in a turn
                elif self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] < 400 :
                    if ship.cell.halite <= 0.0:
                        self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] += 10
                    else:
                        self.ship_actions_bitfield[ship]['actions'][ship.position]['cost'] -= ship.cell.halite

            if should_i_move_towards_shipyard or self.overwhelming_ship_spawn or\
               self.is_none(ship_index) or (ship_index // len(directions)) != 0 :
                ordered_halite_positions = self.get_ordered_halite_positions(ship.position)
            else:
                ordered_halite_positions = self.get_ordered_halite_positions(ship.position, True)
                
            checked_directions = []
            #if self.is_not_none(nearest_shipyard) and self.is_not_none(direction1_nearest_shipyard) and\
            #   self.is_not_none(direction2_nearest_shipyard):
            #    if direction1_nearest_shipyard == direction2_nearest_shipyard:
            #        checked_directions.append(directions[(directions.index(direction1_nearest_shipyard) - 1) % len(directions)])
            #        if distance_nearest_shipyard == 1:
            #            checked_directions.append(direction1_nearest_shipyard)
                    
            for halite_cell in ordered_halite_positions:
                
                if am_i_guard:
                    break
                    
                if self.ship_actions_bitfield[ship]['pirate']:
                    break
                
                if len(checked_directions) == len(directions):
                    break
                    
                if halite_cell in self.assigned_mining_targets and\
                   self.assigned_mining_targets[halite_cell] > self.REUSE_TARGET_LIMIT:
                    continue

                direction1_target = self.get_dir_to(ship.position,\
                                                    Point(halite_cell[0], halite_cell[1]))
                direction2_target = self.get_dir_to_alt(ship.position,\
                                                        Point(halite_cell[0], halite_cell[1]))            

                #if not should_i_spawn:
                    # maintain quadrants
                if self.is_not_none(ship_index) and self.is_not_none(swarm_shipyard):

                    direction1_check = self.get_dir_to(swarm_shipyard.position,\
                                                       Point(halite_cell[0], halite_cell[1]))
                    direction2_check = self.get_dir_to_alt(swarm_shipyard.position,\
                                                           Point(halite_cell[0], halite_cell[1]))

                    if (ship_index % 2 == 0):
                        if directions[ship_index % len(directions)] != direction1_check or\
                           (directions[(ship_index +1) % len(directions)] != direction2_check and\
                            directions[ship_index % len(directions)] != direction2_check):
                            continue
                    else:
                        if directions[ship_index % len(directions)] != direction2_check or\
                           (directions[(ship_index +1) % len(directions)] != direction1_check and\
                            directions[ship_index % len(directions)] != direction1_check):
                            continue
                
                if should_i_move_towards_shipyard or self.overwhelming_ship_spawn or\
                   self.is_none(ship_index) or (ship_index // len(directions)) != 0 :
                    distance, halite = self.cell_halite[halite_cell]
                else:
                    halite, distance = self.cell_halite[halite_cell]
                assigned = False
                if self.is_not_none(direction1_target):
                    if direction1_target not in checked_directions:
                        if self.ship_actions_bitfield[ship]['actions'][(ship.position +\
                                                          direction1_target.to_point())%self.size]['cost'] >= 500:
                            self.ship_actions_bitfield[ship]['actions'][(ship.position +\
                                                              direction1_target.to_point())%self.size]['cost'] += (500 - halite)//distance//4
                            assigned = True
                        checked_directions.append(direction1_target)
                        
                if direction1_target != direction2_target:
                    if self.is_not_none(direction2_target):
                        if direction2_target not in checked_directions:   
                            if self.ship_actions_bitfield[ship]['actions'][(ship.position +\
                                                              direction2_target.to_point())%self.size]['cost'] >= 500:
                                self.ship_actions_bitfield[ship]['actions'][(ship.position +\
                                                                  direction2_target.to_point())%self.size]['cost'] += (500 - halite)//distance//4
                                assigned = True
                            checked_directions.append(direction2_target)
                
                if assigned:
                    if halite_cell in self.assigned_mining_targets:
                        self.assigned_mining_targets[halite_cell] += 1
                    else:
                        self.assigned_mining_targets[halite_cell] = 0

        
            self.position_entries = self.union_of_lists(self.position_entries, \
                                                        self.ship_actions_bitfield[ship]['actions'])
            #print(self.position_entries)
        
        self.ship_action_matrix = np.ones([len(self.ships),len(self.position_entries)], dtype = int) * -50000

        for index, ship in enumerate(self.ships):
            for position in self.ship_actions_bitfield[ship]['actions']:
                self.ship_action_matrix[index, self.position_entries.index(position)] = self.ship_actions_bitfield[ship]['actions'][position]['cost'] 
                    
        #print(self.ship_action_matrix) 
        self.halite_positions_in_new_assignment = {}
        rows, cols = scipy.optimize.linear_sum_assignment(self.ship_action_matrix, maximize = True)
        for row, col in zip(rows,cols):
            if self.position_entries[col] in self.ship_actions_bitfield[self.ships[row]]['actions']:
                self.ships[row].next_action = self.ship_actions_bitfield[self.ships[row]]['actions'][self.position_entries[col]]['action']
                self.halite_positions_in_new_assignment[self.position_entries[col]] = [self.ships[row].halite]
            else:
                self.ship_actions_bitfield[self.ships[row]]['assigned'] = False
                #print("Unassigned")

In [ ]:
%%writefile -a submission.py

    def ship_action_end_move(self):
        
        if self.remaining_shipyard_count == 0:
            return
        
        for ship in self.ships:
            if ship.halite > 0.0:
                nearest_shipyard = self.ship_actions_bitfield[ship]['nearest_shipyard']['shipyard']
                distance_nearest_shipyard = self.ship_actions_bitfield[ship]['nearest_shipyard']['distance']
                direction1_nearest_shipyard = self.ship_actions_bitfield[ship]['nearest_shipyard']['direction1']
                direction2_nearest_shipyard = self.ship_actions_bitfield[ship]['nearest_shipyard']['direction2']
                if self.is_not_none(nearest_shipyard) and\
                   self.is_not_none(direction1_nearest_shipyard) and\
                   self.is_not_none(direction2_nearest_shipyard) :
                    if (distance_nearest_shipyard >= self.episode_steps - self.board.step - 5 and\
                        ship.next_action not in [None, direction1_nearest_shipyard, direction2_nearest_shipyard]) or\
                       (self.episode_steps - self.board.step <= self.size + 2 and\
                        distance_nearest_shipyard >= self.episode_steps - self.board.step - 2 and\
                        ship.next_action not in [direction1_nearest_shipyard, direction2_nearest_shipyard]): 
                        if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction1_nearest_shipyard.to_point())%self.size]['cost'] >= 400:    
                            if (ship.position + direction1_nearest_shipyard.to_point())%self.size in self.halite_positions_in_new_assignment:
                                if ship.halite not in self.halite_positions_in_new_assignment[(ship.position + direction1_nearest_shipyard.to_point())%self.size]:
                                    temp = ship.next_action
                                    ship.next_action = direction1_nearest_shipyard
                                    self.halite_positions_in_new_assignment[(ship.position + direction1_nearest_shipyard.to_point())%self.size].append(ship.halite)
                                    if self.is_none(temp):
                                        self.halite_positions_in_new_assignment[ship.position].remove(ship.halite)
                                    else:
                                        self.halite_positions_in_new_assignment[(ship.position + temp.to_point())%self.size].remove(ship.halite)
                                    continue
                            else:
                                temp = ship.next_action
                                ship.next_action = direction1_nearest_shipyard
                                self.halite_positions_in_new_assignment[(ship.position + direction1_nearest_shipyard.to_point())%self.size] = [ship.halite]
                                if self.is_none(temp):
                                    self.halite_positions_in_new_assignment[ship.position].remove(ship.halite)
                                else:
                                    self.halite_positions_in_new_assignment[(ship.position + temp.to_point())%self.size].remove(ship.halite)
                                continue
                        if self.ship_actions_bitfield[ship]['actions'][(ship.position + direction2_nearest_shipyard.to_point())%self.size]['cost'] >= 400:    
                            if direction1_nearest_shipyard != direction2_nearest_shipyard:
                                if (ship.position + direction2_nearest_shipyard.to_point())%self.size in self.halite_positions_in_new_assignment:
                                    if ship.halite not in self.halite_positions_in_new_assignment[(ship.position + direction2_nearest_shipyard.to_point())%self.size]:
                                        temp = ship.next_action
                                        ship.next_action = direction2_nearest_shipyard
                                        self.halite_positions_in_new_assignment[(ship.position + direction2_nearest_shipyard.to_point())%self.size].append(ship.halite)
                                        if self.is_none(temp):
                                            self.halite_positions_in_new_assignment[ship.position].remove(ship.halite)
                                        else:
                                            self.halite_positions_in_new_assignment[(ship.position + temp.to_point())%self.size].remove(ship.halite)
                                        continue    
                                else:
                                    temp = ship.next_action
                                    ship.next_action = direction2_nearest_shipyard
                                    self.halite_positions_in_new_assignment[(ship.position + direction2_nearest_shipyard.to_point())%self.size] = [ship.halite]
                                    if self.is_none(temp):
                                        self.halite_positions_in_new_assignment[ship.position].remove(ship.halite)
                                    else:
                                        self.halite_positions_in_new_assignment[(ship.position + temp.to_point())%self.size].remove(ship.halite)
                                    continue  

### Routines for Swarm Update and Assignment

In [ ]:
%%writefile -a submission.py

    def get_swarm_shipyard_details(self, ship, shipyard):
        
        swarm_shipyard = None
        ship_index = None
        am_i_guard = False     
        
        if self.is_not_none(shipyard):
            if ship.id in self.my_ships_in_swarm[shipyard.id]:
                swarm_shipyard = shipyard
                ship_index = self.my_ships_in_swarm[shipyard.id].index(ship.id)
                #if ship_index == (len(self.my_ships_in_swarm[shipyard.id]) - 1):
                #    am_i_guard = True
                #ship_index %= len(directions)
                
        return swarm_shipyard, ship_index, am_i_guard

In [ ]:
%%writefile -a submission.py

    def get_nearest_shipyard_details(self, ship, shipyard_id_list):
        
        nearest_shipyard = None
        distance_nearest_shipyard = self.size * 2
        direction1_nearest_shipyard = None
        direction2_nearest_shipyard = None
        
        nearest_shipyard = self.get_nearest_entity(ship.position, shipyard_id_list)

        if self.is_not_none(nearest_shipyard):
            
            distance_nearest_shipyard = self.get_step_distance(ship.position, nearest_shipyard.position)
            direction1_nearest_shipyard = self.get_dir_to(ship.position, nearest_shipyard.position)
            direction2_nearest_shipyard = self.get_dir_to_alt(ship.position, nearest_shipyard.position)

        return nearest_shipyard, distance_nearest_shipyard, direction1_nearest_shipyard, direction2_nearest_shipyard

In [ ]:
%%writefile -a submission.py

    def update_swarm_info(self):
               
        self.my_ships_in_swarm = {}
        
        if len(self.shipyards) == 0 or len(self.ships) == 0:
            return
        
        tmp_shipyards = {}
        for shipyard_id in self.shipyard_ids:
            tmp_shipyards[shipyard_id] = {}
            for direction in directions:
                tmp_shipyards[shipyard_id][direction] = {}  
                tmp_shipyards[shipyard_id]['Guard'] = None
                
        #if self.me.id == 0 :print(self.my_ships_in_swarm)
        for ship in self.ships:
            nearest_shipyard = None
            distance_nearest_shipyard = self.size * 2
            direction1_nearest_shipyard = None
            direction2_nearest_shipyard = None             
            
            nearest_shipyard, distance_nearest_shipyard, direction1_nearest_shipyard, \
            direction2_nearest_shipyard = self.get_nearest_shipyard_details(ship, self.shipyard_ids)
            
            if self.is_not_none(nearest_shipyard):                
                if directions[0] == direction1_nearest_shipyard and\
                   (directions[1] == direction2_nearest_shipyard or\
                    directions[0] == direction2_nearest_shipyard):
                    #if self.me.id == 0 :print(self.board.step, ship.position, distance_nearest_shipyard, directions[2])
                    tmp_shipyards[nearest_shipyard.id][directions[2]][ship.id] = distance_nearest_shipyard 
                elif directions[1] == direction2_nearest_shipyard and\
                     (directions[2] == direction1_nearest_shipyard or\
                      directions[1] == direction1_nearest_shipyard):
                    #if self.me.id == 0 :print(self.board.step, ship.position, distance_nearest_shipyard, directions[3])
                    tmp_shipyards[nearest_shipyard.id][directions[3]][ship.id] = distance_nearest_shipyard 
                elif directions[2] == direction1_nearest_shipyard and\
                     (directions[3] == direction2_nearest_shipyard or\
                      directions[2] == direction2_nearest_shipyard):
                    #if self.me.id == 0 :print(self.board.step, ship.position, distance_nearest_shipyard, directions[0])
                    tmp_shipyards[nearest_shipyard.id][directions[0]][ship.id] = distance_nearest_shipyard 
                elif directions[3] == direction2_nearest_shipyard and\
                     (directions[0] == direction1_nearest_shipyard or\
                      directions[3] == direction1_nearest_shipyard):
                    #if self.me.id == 0 :print(self.board.step, ship.position, distance_nearest_shipyard, directions[1])
                    tmp_shipyards[nearest_shipyard.id][directions[1]][ship.id] = distance_nearest_shipyard 
                else:
                    tmp_shipyards[nearest_shipyard.id]['Guard'] = ship.id
        
        for shipyard_id in tmp_shipyards:
            guard_index = False
            if self.is_none(tmp_shipyards[shipyard_id]['Guard']):
                distance = self.size ** 2
                tmp_directions = []
                min_len = 0
                for direction in directions:
                    if len(tmp_shipyards[shipyard_id][direction]) > min_len:
                        tmp_directions = [direction]
                        min_len = len(tmp_shipyards[shipyard_id][direction])
                    elif len(tmp_shipyards[shipyard_id][direction]) == min_len:
                        tmp_directions.append(direction)
                        
                for direction in tmp_directions :
                    for ship_id in tmp_shipyards[shipyard_id][direction]:
                        dist = tmp_shipyards[shipyard_id][direction][ship_id]
                        if dist < distance:
                            guard_index = True
                            guard_direction = direction
                            distance = dist
                            guard_ship_id = ship_id
            if guard_index:
                tmp_shipyards[shipyard_id][guard_direction].pop(guard_ship_id)
                tmp_shipyards[shipyard_id]['Guard'] = guard_ship_id
            elif self.is_not_none(tmp_shipyards[shipyard_id]['Guard']):
                # set guard ship direction to one having least no of ships
                length = self.remaining_ship_count + 1
                check = []
                for direction in directions:
                    temp = len(tmp_shipyards[shipyard_id][direction])
                    if temp == length:
                        check.append(direction)
                    elif temp < length:
                        check = [direction]
                        guard_index = True
                        guard_direction = direction
                        length = temp
                #if self.me.id == 0 :print(self.board.step, check, length, tmp_shipyards[shipyard_id][guard_direction])
                if len(check) > 1:
                    # same no of ships in all directions
                    guard_ship = self.ships[self.ship_ids.index(tmp_shipyards[shipyard_id]['Guard'])]
                    ordered_halite_positions = self.get_ordered_halite_positions(guard_ship.position, True)
                    for halite_cell in ordered_halite_positions:
                        direction1_target = self.get_dir_to(guard_ship.position,\
                                                Point(halite_cell[0], halite_cell[1]))
                        direction2_target = self.get_dir_to_alt(guard_ship.position,\
                                                    Point(halite_cell[0], halite_cell[1])) 
                        if directions[0] == direction1_target and\
                           (directions[1] == direction2_target or\
                            directions[0] == direction2_target):
                            if directions[0] in check:
                                guard_direction = directions[0]
                                break
                        elif directions[1] == direction2_target and\
                             (directions[2] == direction1_target or\
                              directions[1] == direction1_target):
                            if directions[1] in check:
                                guard_direction = directions[1]
                                break
                        elif directions[2] == direction1_target and\
                             (directions[3] == direction2_target or\
                              directions[2] == direction2_target):
                            if directions[2] in check:
                                guard_direction = directions[2]
                                break
                        elif directions[3] == direction2_target and\
                             (directions[0] == direction1_target or\
                             directions[3] == direction1_target):
                            if directions[3] in check:
                                guard_direction = directions[3]
                                break
                            
            self.my_ships_in_swarm[shipyard_id] = []
            for direction in directions:
                tmp_shipyards[shipyard_id][direction] = sorted(tmp_shipyards[shipyard_id][direction],\
                                                               key = tmp_shipyards[shipyard_id][direction].get,\
                                                               reverse = True)
            
            while True:
                finished = True
                for direction in directions:                    
                    if len(tmp_shipyards[shipyard_id][direction]) != 0:
                        if len(tmp_shipyards[shipyard_id][direction]) != 1:
                            finished = False
                        while len(self.my_ships_in_swarm[shipyard_id])%len(directions) != directions.index(direction):
                            self.my_ships_in_swarm[shipyard_id].append("DUMMY")
                        ship_id = tmp_shipyards[shipyard_id][direction][0]
                        self.my_ships_in_swarm[shipyard_id].append(ship_id)
                        tmp_shipyards[shipyard_id][direction].remove(ship_id)
                        continue
                if finished:
                    if self.is_not_none(tmp_shipyards[shipyard_id]['Guard']):
                        if guard_index:
                            while len(self.my_ships_in_swarm[shipyard_id])%len(directions) != directions.index(guard_direction):
                                self.my_ships_in_swarm[shipyard_id].append("DUMMY")
                        self.my_ships_in_swarm[shipyard_id].append(tmp_shipyards[shipyard_id]['Guard'])    
                    break
                    
        #if self.me.id == 0 :
            #print(self.my_ships_in_swarm)          

### Detect Termination Conditions

In [ ]:
%%writefile -a submission.py
    
    def convert_now(self):
        convert_now_count = 0
        play_over_count = 0
        for opponent in self.board.opponents:
            shipyard_count = 0
            ship_count = 0
            for shipyard in opponent.shipyards:
                shipyard_count += 1
            for ship in opponent.ships:
                ship_count += 1
            if opponent.halite < self.convert_cost and ship_count ==0:    
                play_over_count += 1
            elif opponent.halite < self.convert_cost and (shipyard_count == 0 or ship_count <= 1):#assuming he will not be able to get enough halite 500+ to convert and spawn with any remaining ships, though it is a valid possibility
                convert_now_count += 1
        return (play_over_count == 2 and convert_now_count == 1)

### Exploration Functions

In [ ]:
%%writefile -a submission.py

    def generate_farming_map(self):
        halite = np.array(self.board.observation['halite']).reshape(self.size, self.size).astype(int)
        halite[halite.nonzero()] = 1
        farm_filter = np.array([[0,0,1,0,0],[0,1,1,1,0],[1,1,0,1,1],[0,1,1,1,0],[0,0,1,0,0]])
        farm_array = scipy.ndimage.convolve(halite, farm_filter, mode='wrap', cval=0.0)
        self.farming_targets = []
        for i in range(0,6):
            if np.where(farm_array >= 12 - i, 1, 0).nonzero()[0].size == 0:
                continue
            if self.me.id == 0: print(np.where(farm_array >= 12 - i, 1, 0).nonzero())
            targets = np.logical_and(np.where(farm_array >= 12 - i, 1, 0),np.logical_not(halite)).astype(int).nonzero()
            for index in range(targets[0].size):
                self.farming_targets.append(Point(targets[1][index], self.size -1 - targets[0][index]))
            if len(self.farming_targets) != 0:
                break
        if self.me.id == 0: print(self.farming_targets)
        #targets = np.logical_and(np.where(farm_array == 3, 1, 0),np.logical_not(halite)).astype(int).nonzero()
        #for index in range(targets[0].size):
        #    self.farming_targets.append(Point(targets[1][index], self.size -1 - targets[0][index]))
        #if self.me.id == 0: print(self.farming_targets)
        
    def update_halite_map(self):
        self.halite = np.array(self.board.observation['halite']).reshape(self.size, self.size)
        a = self.halite.nonzero()
        # Global Halite Threshold
        self.HALITE_THRESHOLD = np.median(pd.Series(self.halite[a]))
        #if self.board.step <= self.episode_steps // 8:
        #    self.halite[self.halite < min(2 * self.HALITE_THRESHOLD_MIN, 2 * self.HALITE_THRESHOLD)] = 0.0
        #else:
        self.halite[self.halite < min(self.HALITE_THRESHOLD_MIN, self.HALITE_THRESHOLD)] = 0.0
        a = self.halite.nonzero()
        # Local Halite Threshold
        self.HALITE_THRESHOLD_LOCAL = np.median(pd.Series(self.halite[a]))  
        #if self.board.step <= self.episode_steps * 3 // 4 :
        self.halite[self.halite < self.HALITE_THRESHOLD_LOCAL] = 0.0
        self.mining_targets = self.halite.nonzero()
        
    def get_ordered_halite_positions(self, position, max_ordering = False):
        
        # decide seed ship actions
        self.cell_halite = {}
        for index in range(self.mining_targets[0].size):
            distance = self.get_step_distance(Point(self.mining_targets[1][index],self.size -1 - self.mining_targets[0][index]), position)
            if distance < 2:
                continue
            #if not max_ordering:
            #    nearest_shipyard = self.get_nearest_entity(Point(self.mining_targets[1][index], \
            #                                                     self.size -1 - self.mining_targets[0][index]), \
            #                                               self.shipyard_ids)
            #    if self.is_not_none(nearest_shipyard):
            #        distance_nearest_shipyard = self.get_step_distance(Point(self.mining_targets[1][index], \
            #                                                                self.size -1 - self.mining_targets[0][index]), \
            #                                                           nearest_shipyard.position)
            #        distance += distance_nearest_shipyard
            #if distance > self.size//2 or distance < 2:
                #continue
            if max_ordering :
                self.cell_halite[self.mining_targets[1][index], self.size -1 - self.mining_targets[0][index]] = \
                round(500.0-self.halite[self.mining_targets[0][index],self.mining_targets[1][index]]), distance
            else:
                self.cell_halite[self.mining_targets[1][index], self.size -1 - self.mining_targets[0][index]] = \
                distance, round(500.0-self.halite[self.mining_targets[0][index],self.mining_targets[1][index]])
                
        #print(cell_halite)    
        return sorted(self.cell_halite, key = self.cell_halite.get)#, reverse=True)

In [ ]:
%%writefile -a submission.py

    def create_distance_lookup(self):
        for index1 in range(self.size):
            for index2 in range(self.size):
                self.reference_distance[(index1, index2)] = {}
        for reference in self.reference_distance:
            for index1 in range(self.size):
                for index2 in range(self.size):
                    if (index1, index2) in self.reference_distance:
                        if reference not in self.reference_distance[(index1, index2)]:
                            self.reference_distance[reference][(index1, index2)] = self.get_step_distance_init(Point(reference[0], reference[1]),Point(index1, index2))
                    else:
                        self.reference_distance[reference][(index1, index2)] = self.get_step_distance_init(Point(reference[0], reference[1]), Point(index1, index2))

    def get_step_distance(self, A, B):
        if (B.x,B.y) in self.reference_distance[(A.x,A.y)]:
            return self.reference_distance[(A.x,A.y)][(B.x,B.y)]
        else:
            return self.reference_distance[(B.x,B.y)][(A.x,A.y)]

    def get_step_distance_init(self, A, B):
        a = abs(A.x - B.x)
        b = abs(A.y - B.y)
        if a > self.size//2:
            a = (self.size - abs(A.x - B.x))
        if b > self.size//2:
            b = (self.size - abs(A.y - B.y))
        return a + b

In [ ]:
%%writefile -a submission.py
    
    # Returns best direction to move from one position (fromPos) to another (toPos)
    # Example: If I'm at pos 0 and want to get to pos 55, which direction should I choose?
    def get_dir_to(self, fromPos, toPos):
        fromPos = fromPos % self.size
        toPos = toPos % self.size
        if (fromPos.y < toPos.y and toPos.y - fromPos.y <= self.size//2) or (fromPos.y > toPos.y and fromPos.y - toPos.y > self.size//2): 
            return ShipAction.NORTH
        if (fromPos.y > toPos.y and fromPos.y - toPos.y <= self.size//2) or (fromPos.y < toPos.y and toPos.y - fromPos.y > self.size//2): 
            return ShipAction.SOUTH
        if (fromPos.x < toPos.x and toPos.x - fromPos.x <= self.size//2) or (fromPos.x > toPos.x and fromPos.x - toPos.x > self.size//2): 
            return ShipAction.EAST        
        if (fromPos.x > toPos.x and fromPos.x - toPos.x <= self.size//2) or (fromPos.x < toPos.x and toPos.x - fromPos.x > self.size//2): 
            return ShipAction.WEST
        return None

    # Returns best direction to move from one position (fromPos) to another (toPos)
    # Example: If I'm at pos 0 and want to get to pos 55, which direction should I choose?
    def get_dir_to_alt(self, fromPos, toPos):
        fromPos = fromPos % self.size
        toPos = toPos % self.size
        if (fromPos.x < toPos.x and toPos.x - fromPos.x <= self.size//2) or (fromPos.x > toPos.x and fromPos.x - toPos.x > self.size//2): 
            return ShipAction.EAST        
        if (fromPos.x > toPos.x and fromPos.x - toPos.x <= self.size//2) or (fromPos.x < toPos.x and toPos.x - fromPos.x > self.size//2): 
            return ShipAction.WEST
        if (fromPos.y < toPos.y and toPos.y - fromPos.y <= self.size//2) or (fromPos.y > toPos.y and fromPos.y - toPos.y > self.size//2): 
            return ShipAction.NORTH
        if (fromPos.y > toPos.y and fromPos.y - toPos.y <= self.size//2) or (fromPos.y < toPos.y and toPos.y - fromPos.y > self.size//2): 
            return ShipAction.SOUTH
        return None

### Helper Functions

In [ ]:
%%writefile -a submission.py
    
    def is_not_none(self, entity_id):
        if entity_id is not None:
            return True
        return False                          
    
    def is_none(self, entity_id):
        return not self.is_not_none(entity_id) 

In [ ]:
%%writefile -a submission.py
    
    def intersection_of_lists(self, list1, list2):
        return list(set(list1).intersection(list2))

    def union_of_lists(self, list1, list2):
        return list(set(list1).union(list2))

In [ ]:
%%writefile -a submission.py
    
    def get_nearest_entity(self, from_entity_position, to_entity_id_list):
        dist = self.size * 2
        selected_entity = None
        for entity_id in to_entity_id_list:
            entity = self.find_entity_by_id(entity_id)
            if self.is_not_none(entity):
                temp = self.get_step_distance(from_entity_position, entity.position)
                if temp < dist:
                    dist = temp
                    selected_entity = entity
        # if two shipyards are at same distance, it will select the first one,
        # this may not always be an optimal solution
        return selected_entity 
    
    def find_entity_by_id(self, entity_id):
        if entity_id in self.ship_ids:
            return self.ships[self.ship_ids.index(entity_id)]
        elif entity_id in self.shipyard_ids:
            return self.shipyards[self.shipyard_ids.index(entity_id)]
        return None

In [ ]:
%%writefile -a submission.py
    
    def get_my_ships(self, list_of_ships):
        return self.intersection_of_lists(list_of_ships, self.ships)    

    def get_my_shipyards(self, list_of_shipyards):
        return self.intersection_of_lists(list_of_shipyards, self.shipyards)  
    
    def is_my_ship(self, ship):
        if ship in self.ships:
            return True
        return False   

    def is_my_shipyard(self, shipyard):
        if shipyard in self.shipyards:
            return True
        return False

In [ ]:
%%writefile -a submission.py    
    
    def get_my_ships_by_id(self, list_of_ship_ids):
        my_ship_ids = []
        for ship_id in list_of_ship_ids:
            if self.is_my_ship_by_id(ship_id):
                my_ship_ids.append(ship_id)
        return my_ship_ids  

    def is_my_ship_by_id(self, ship_id):
        if ship_id in self.ship_ids:
            return True
        return False     

In [ ]:
%%writefile -a submission.py
    
    def get_my_shipyards_by_id(self, list_of_shipyard_ids):
        my_shipyard_ids = []
        for shipyard_id in list_of_shipyard_ids:
            if self.is_my_shipyard_by_id(shipyard_id):
                my_shipyard_ids.append(shipyard_id)
        return my_shipyard_ids  
    
    def is_my_shipyard_by_id(self, shipyard_id):
        if shipyard_id in self.shipyard_ids:
            return True
        return False       

In [ ]:
%%writefile -a submission.py    

    def get_enemy_ships(self, list_of_ships):
        return self.intersection_of_lists(list_of_ships, self.enemy_ships)  

    def get_enemy_shipyards(self, list_of_shipyards):
        return self.intersection_of_lists(list_of_shipyards, self.enemy_shipyards)
    
    def is_enemy_ship(self, ship):
        if ship in self.enemy_ships:
            return True
        return False 
    
    def is_enemy_shipyard(self, shipyard):
        if shipyard in self.enemy_shipyards:
            return True
        return False     

In [ ]:
%%writefile -a submission.py

    def get_adjacent_cell_in_direction(self, cell, direction):
        ret = None
        if direction == ShipAction.NORTH:
            ret = cell.north
        elif direction == ShipAction.EAST:
            ret = cell.east
        elif direction == ShipAction.WEST:
            ret = cell.west
        elif direction == ShipAction.SOUTH:
            ret = cell.south 
        return ret 
    
    def get_adjacent_halite_in_direction(self, cell, direction):
        cell_ret = self.get_adjacent_cell_in_direction(cell, direction)
        if self.is_not_none(cell_ret):
            return cell_ret.halite
        return 0.0 

    def get_adjacent_ship_in_direction(self, cell, direction):
        cell_ret = self.get_adjacent_cell_in_direction(cell, direction)
        if self.is_not_none(cell_ret):
            return cell_ret.ship
        return None   
    
    def get_adjacent_shipyard_in_direction(self, cell, direction):
        cell_ret = self.get_adjacent_cell_in_direction(cell, direction)
        if self.is_not_none(cell_ret):
            return cell_ret.shipyard
        return None     

    def get_nearby_ships_in_direction(self, cell, direction):
        adj_cell = self.get_adjacent_cell_in_direction(cell, direction)
        adj = []
        if self.is_not_none(adj_cell):
            adj = self.get_adjacent_ships(adj_cell)
        if cell.ship in adj:
            adj.remove(cell.ship) 
        return list(set(adj))
    
    def get_nearby_shipyards_in_direction(self, cell, direction):
        adj_cell = self.get_adjacent_cell_in_direction(cell, direction)
        adj = []
        if self.is_not_none(adj_cell):
            adj = self.get_adjacent_shipyards(adj_cell)
        if cell.shipyard in adj:
            adj.remove(cell.shipyard) 
        return list(set(adj))
    
    def get_approaching_ships_in_direction(self, cell, direction):    
        adj = []
        adj_cell = self.get_adjacent_cell_in_direction(cell, direction)
        if self.is_not_none(adj_cell):
            adj.extend(self.get_nearby_ships(adj_cell)) 
        for direction in directions:
            adj_cell = self.get_adjacent_cell_in_direction(cell, direction)
            if adj_cell.ship in adj:
                adj.remove(adj_cell.ship) 
        return list(set(adj))

In [ ]:
%%writefile -a submission.py

    def get_adjacent_ships(self, cell):
        adj = []
        for direction in directions:    
            adj_ship = self.get_adjacent_ship_in_direction(cell, direction)        
            if self.is_not_none(adj_ship):
                adj.append(adj_ship)
        return adj
 
    def get_adjacent_shipyards(self, cell):
        adj = []
        for direction in directions:    
            adj_shipyard = self.get_adjacent_shipyard_in_direction(cell, direction)        
            if self.is_not_none(adj_shipyard):
                adj.append(adj_shipyard)
        return adj
    
    def get_nearby_ships(self, cell):
        adj = []
        for direction in directions:
            adj_cell = self.get_adjacent_cell_in_direction(cell, direction)
            if self.is_not_none(adj_cell):
                adj.extend(self.get_adjacent_ships(adj_cell))
        if cell.ship in adj:
            adj.remove(cell.ship) 
        return list(set(adj))

    def get_nearby_shipyards(self, cell):
        adj = []
        for direction in directions:
            adj_cell = self.get_adjacent_cell_in_direction(cell, direction)
            if self.is_not_none(adj_cell):
                adj.extend(self.get_adjacent_shipyards(adj_cell))
        if cell.shipyard in adj:
            adj.remove(cell.shipyard) 
        return list(set(adj))
    
    def get_approaching_ships(self, cell):    
        adj = []
        for direction in directions:
            adj_cell = self.get_adjacent_cell_in_direction(cell, direction)
            if self.is_not_none(adj_cell):
                adj.extend(self.get_nearby_ships(adj_cell))
        if cell.ship in adj:
            adj.remove(cell.ship) 
        for direction in directions:
            adj_cell = self.get_adjacent_cell_in_direction(cell, direction)
            if adj_cell.ship in adj:
                adj.remove(adj_cell.ship) 
        return list(set(adj))

## Define Agent

In [ ]:
%%writefile -a submission.py

def agent(obs, config):
    global init_Controller
    global controller
    if not init_Controller:
        controller = Controller(obs, config) 
        init_Controller = True
    return controller.next_actions(obs, config)

### EOF